In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pickle
from PIL import Image
import adapt
from adapt.feature_based import DANN, CORAL, DeepCORAL, CDAN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

In [2]:
path = "I:/Temp/CS 766 CCPD Data"
dataset_path = { "base": path + "/ccpd/splitted_plates_base",
                "challenge": path + "/ccpd/splitted_plates_challenge",
               "db": path + "/ccpd/splitted_plates_db",
               "fn": path + "/ccpd/splitted_plates_fn",
               "weather": path + "/ccpd/splitted_plates_weather"}
img_h, img_w = 32,16

In [ ]:
def get_Xy():

    X = []
    y = []

    for r, d, f in os.walk(path):
        for direct in d:
            if not ".ipynb_checkpoints" in direct:
                for r, d, f in os.walk(os.path.join(path , direct)):
                    for file in f:
                        path_to_image = os.path.join(r, file)
                        if not ".ipynb_checkpoints" in path_to_image:
                            image = Image.open(path_to_image)
                            image = image.resize((32, 16), Image.ANTIALIAS)
                            image = np.array(image, dtype=int)
                            X.append(image)
                            y.append(direct)
    return X, y

Xs, ys = get_Xy()

In [3]:
def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys():
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label)
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = Image.open(img_dir)
#         img = img.convert('L')# conver to grayscale images
#         img = img.resize([img_w, img_h])
#         img_np = np.asarray(img)
#         X.append(img_np.reshape([-1]))
#     X = np.array(X)
#     return X

def readCcpdImg_noreshape(images_dir):
    X = []
    for img_dir in images_dir:
        img = Image.open(img_dir)
        img = img.convert('RGB') # conver to grayscale images
        img = img.resize([img_w, img_h],Image.ANTIALIAS)
        img_np = np.asarray(img)
        X.append(img_np)
    X = np.array(X)
    return X

# def get_img_label(path, mode,num_classes=35):
#     images_dir, labels, table = load_ccpd(dataset_path[path],mode=mode)
#     images = readCcpdImg(images_dir)
#     labels = np.array(labels)
#     labels = to_categorical(labels, num_classes=num_classes)
#     return images,labels

def get_img_label_noreshape(path, mode,num_classes=35):
    images_dir, labels, table = load_ccpd(dataset_path[path],mode=mode)
    images = readCcpdImg_noreshape(images_dir)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    return images,labels

In [4]:
def get_encoder(input_shape=(32,16,3)):
    model = Sequential()
    model.add(Convolution2D(32,(3,3),activation='relu', padding='valid', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Convolution2D(32,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(720, activation="relu"))
    model.add(Dense(168*3, activation="relu"))
    return model

def get_task(input_shape=(168*3,)):
    model = Sequential()
    model.add(Dense(168*3, activation='relu',input_shape=input_shape))
    model.add(Dropout(0.5))
    model.add(Dense(34, activation="softmax"))
    return model

def get_discriminator(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [8]:
base_images,base_labels = get_img_label_noreshape('base', 'all', num_classes=34)

In [9]:
weather_images_t,weather_labels_t = get_img_label_noreshape('weather', 'train', num_classes=34)

In [10]:
from adapt.feature_based import CCSA

weather_model_CCSA = CCSA(encoder=get_encoder(), task=get_task(), Xt=weather_images_t, yt=weather_labels_t, optimizer=Adam(0.001), loss='CategoricalCrossentropy', metrics=["acc"],random_state=0)
weather_model_CCSA.fit(X=base_images, y=base_labels, Xt=weather_images_t, yt=weather_labels_t, epochs=45,verbose=1,batch_size = 32)
weather_model_CCSA.score(base_images, base_labels)
# weather_model_CCSA.score(weather_images, weather_labels)

Epoch 1/60
286/286 [==============================] - 7s 15ms/step - loss: 2.9092 - acc: 0.2825 - contrast: 91.9813
Epoch 2/60
286/286 [==============================] - 4s 14ms/step - loss: 1.9572 - acc: 0.4335 - contrast: 0.3454
Epoch 3/60
286/286 [==============================] - 4s 14ms/step - loss: 1.5898 - acc: 0.5092 - contrast: 0.2951
Epoch 4/60
286/286 [==============================] - 4s 14ms/step - loss: 1.3283 - acc: 0.5868 - contrast: 0.2652
Epoch 5/60
286/286 [==============================] - 4s 14ms/step - loss: 1.1213 - acc: 0.6460 - contrast: 0.2414
Epoch 6/60
286/286 [==============================] - 4s 14ms/step - loss: 1.0003 - acc: 0.6867 - contrast: 0.2211
Epoch 7/60
286/286 [==============================] - 4s 15ms/step - loss: 0.8613 - acc: 0.7359 - contrast: 0.2153
Epoch 8/60
286/286 [==============================] - 4s 15ms/step - loss: 0.7294 - acc: 0.7806 - contrast: 0.1898
Epoch 9/60
286/286 [==============================] - 4s 15ms/step - loss: 0.64

0.025460468605160713

In [11]:
weather_images,weather_labels = get_img_label_noreshape('weather', 'all', num_classes=34)
weather_model_CCSA.score(weather_images, weather_labels)

1/1 [==============================] - 1s 600ms/step - loss: 0.1293 - acc: 0.9746


0.1292838752269745

In [12]:
save_check_pt = 'checkpoints_CCSA'

In [13]:
weather_model_CCSA.save_weights(save_check_pt+ '/weather')

In [14]:
challenge_images_t,challenge_labels_t = get_img_label_noreshape('challenge', 'train', num_classes=34)
challenge_model_CCSA = CCSA(encoder=get_encoder(), task=get_task(), Xt=challenge_images_t, yt=challenge_labels_t, optimizer=Adam(0.001), loss='CategoricalCrossentropy', metrics=["acc"],random_state=0)

In [15]:
challenge_model_CCSA.fit(X=base_images, y=base_labels, Xt=challenge_images_t, yt=challenge_labels_t, epochs=60,verbose=1,batch_size = 32)
challenge_model_CCSA.score(base_images, base_labels)

Epoch 1/60
266/266 [==============================] - 5s 15ms/step - loss: 2.8951 - acc: 0.3097 - contrast: 171.9930
Epoch 2/60
266/266 [==============================] - 4s 14ms/step - loss: 1.9070 - acc: 0.5093 - contrast: 0.4240
Epoch 3/60
266/266 [==============================] - 4s 14ms/step - loss: 1.4874 - acc: 0.6039 - contrast: 0.3669
Epoch 4/60
266/266 [==============================] - 4s 14ms/step - loss: 1.2843 - acc: 0.6584 - contrast: 0.3403
Epoch 5/60
266/266 [==============================] - 4s 14ms/step - loss: 1.1182 - acc: 0.6935 - contrast: 0.3096
Epoch 6/60
266/266 [==============================] - 4s 14ms/step - loss: 1.0346 - acc: 0.7114 - contrast: 0.2843
Epoch 7/60
266/266 [==============================] - 4s 14ms/step - loss: 0.9637 - acc: 0.7295 - contrast: 0.2572
Epoch 8/60
266/266 [==============================] - 4s 14ms/step - loss: 0.8868 - acc: 0.7537 - contrast: 0.2382
Epoch 9/60
266/266 [==============================] - 4s 14ms/step - loss: 0.8

0.04681152105331421

In [16]:
challenge_images,challenge_labels = get_img_label_noreshape('challenge', 'all', num_classes=34)
challenge_model_CCSA.score(challenge_images, challenge_labels)

1/1 [==============================] - 0s 230ms/step - loss: 0.6546 - acc: 0.8504


0.6545549631118774

In [17]:
challenge_model_CCSA.save_weights(save_check_pt+ '/challenge')

This is db

In [18]:
db_images_t,db_labels_t = get_img_label_noreshape('db', 'train', num_classes=34)
db_model_CCSA = CCSA(encoder=get_encoder(), task=get_task(), Xt=db_images_t, yt=db_labels_t, optimizer=Adam(0.001), loss='CategoricalCrossentropy', metrics=["acc"],random_state=0)

In [19]:
db_model_CCSA.fit(X=base_images, y=base_labels, Xt=db_images_t, yt=db_labels_t, epochs=60,verbose=1,batch_size = 32)
db_model_CCSA.score(base_images, base_labels)

Epoch 1/60
269/269 [==============================] - 5s 15ms/step - loss: 3.1816 - acc: 0.2513 - contrast: 159.0702
Epoch 2/60
269/269 [==============================] - 4s 14ms/step - loss: 2.5484 - acc: 0.3470 - contrast: 0.4933
Epoch 3/60
269/269 [==============================] - 4s 14ms/step - loss: 2.0519 - acc: 0.4542 - contrast: 0.4156
Epoch 4/60
269/269 [==============================] - 4s 14ms/step - loss: 1.6959 - acc: 0.5455 - contrast: 0.3781
Epoch 5/60
269/269 [==============================] - 4s 14ms/step - loss: 1.4721 - acc: 0.6018 - contrast: 0.3506
Epoch 6/60
269/269 [==============================] - 4s 15ms/step - loss: 1.3405 - acc: 0.6383 - contrast: 0.3279
Epoch 7/60
269/269 [==============================] - 4s 14ms/step - loss: 1.2566 - acc: 0.6572 - contrast: 0.2995
Epoch 8/60
269/269 [==============================] - 4s 14ms/step - loss: 1.1401 - acc: 0.6845 - contrast: 0.2768
Epoch 9/60
269/269 [==============================] - 4s 14ms/step - loss: 1.0

0.04010641947388649

In [20]:
db_images,db_labels = get_img_label_noreshape('db', 'all', num_classes=34)
db_model_CCSA.score(db_images, db_labels)

1/1 [==============================] - 1s 524ms/step - loss: 0.5229 - acc: 0.8882


0.5229292511940002

In [21]:
db_model_CCSA.save_weights(save_check_pt+ '/db')

This is fn

In [22]:
fn_images_t,fn_labels_t = get_img_label_noreshape('fn', 'train', num_classes=34)
fn_model_CCSA = CCSA(encoder=get_encoder(), task=get_task(), Xt=fn_images_t, yt=fn_labels_t, optimizer=Adam(0.001), loss='CategoricalCrossentropy', metrics=["acc"],random_state=0)

In [23]:
fn_model_CCSA.fit(X=base_images, y=base_labels, epochs=60,verbose=1,batch_size = 32) #, Xt=fn_images_t, yt=fn_labels_t
fn_model_CCSA.score(base_images, base_labels)

Epoch 1/60
264/264 [==============================] - 5s 14ms/step - loss: 2.9154 - acc: 0.3034 - contrast: 157.5751
Epoch 2/60
264/264 [==============================] - 4s 14ms/step - loss: 1.8762 - acc: 0.4947 - contrast: 0.2947
Epoch 3/60
264/264 [==============================] - 4s 14ms/step - loss: 1.4566 - acc: 0.5922 - contrast: 0.2477
Epoch 4/60
264/264 [==============================] - 4s 14ms/step - loss: 1.1884 - acc: 0.6615 - contrast: 0.2305
Epoch 5/60
264/264 [==============================] - 4s 14ms/step - loss: 1.0393 - acc: 0.7143 - contrast: 0.2146
Epoch 6/60
264/264 [==============================] - 4s 14ms/step - loss: 0.9057 - acc: 0.7549 - contrast: 0.2021
Epoch 7/60
264/264 [==============================] - 4s 14ms/step - loss: 0.8117 - acc: 0.7836 - contrast: 0.1882
Epoch 8/60
264/264 [==============================] - 4s 14ms/step - loss: 0.7171 - acc: 0.8004 - contrast: 0.1723
Epoch 9/60
264/264 [==============================] - 4s 14ms/step - loss: 0.6

0.05891306698322296

In [24]:
fn_images,fn_labels = get_img_label_noreshape('fn', 'all', num_classes=34)
fn_model_CCSA.score(fn_images, fn_labels)

1/1 [==============================] - 0s 280ms/step - loss: 0.1205 - acc: 0.9794


0.12048079073429108

In [25]:
fn_model_CCSA.score(base_images, base_labels)

1/1 [==============================] - 0s 243ms/step - loss: 0.0589 - acc: 0.9846


0.05891306698322296

In [26]:
fn_model_CCSA.save_weights(save_check_pt+ '/fn')